In [1]:
import multiprocessing
import os
import sys
from sklearn.model_selection import train_test_split

sys.path.append("..")

from movingpose.estimator import neighbors
from movingpose.estimator import classifiers

from movingpose.preprocessing import moving_pose
from movingpose.preprocessing import kinect_skeleton_data

## Pickle multiview action data

In [2]:
kinect_skeleton_data.pickle_dir("../pickle/multiview.p", "../ext/dataset/multiview_action/")

Pickling skeleton data...
Saving data...
Saved data to file: ../pickle/multiview.p
Program took 16.7268705368042 to run!


## Load pickled multiview action data

In [3]:
raw_data_dict = kinect_skeleton_data.load_pickle("../pickle/multiview.p")

## Format multiview action data

In [4]:
X, labels = moving_pose.format_skeleton_data_dict(raw_data_dict)

## Perform basic verification that skeleton data is formatted correctly

In [5]:
for action in X:
    for frame_num, pose in enumerate(action):
        assert pose.shape == (20, 10), f"{pose.shape} =/= (20, 10)"
        for i, descriptor in enumerate(pose):
            if i == 0:
                assert list(descriptor)[:-1] == [0, 0, 0, 0, 0, 0, 0, 0, 0]
            assert descriptor[-1] == frame_num, f"{descriptor[-1]} =/= {frame_num}"

## Create train/test split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, random_state=42)

## Pickle Action Classifiers

In [7]:
def pickle_action_classifiers(n_training_neighbors, cache_path):
    nearest_pose_estimator = neighbors.NearestPoses(
        n_neighbors=20,
        n_training_neighbors=n_training_neighbors,
        alpha=0.75,
        beta=0.6,
        kappa=80
    )
    action_classifier = classifiers.ActionClassifier(
        nearest_pose_estimator=nearest_pose_estimator,
        theta=0.5,
        n=80
    )
    action_classifier.fit(
        X_train,
        y_train,
        cache_path=cache_path,
        verbose=True
    )

cache_workers = []
for n_training_neighbors in [2000, 5000, 10000, 20000, 30000, 40000]:
    cache_path = f"../pickle/action_classifier_training-{str(n_training_neighbors)}.p"
    if not os.path.exists(cache_path):
        worker = multiprocessing.Process(
            target=
                pickle_action_classifiers,
            args=(
                n_training_neighbors,
                cache_path
            )
        )
        cache_workers.append(worker)

for cache_worker in cache_workers:
    cache_worker.start()
    
for cache_worker in cache_workers:
    cache_worker.join()

## Train Action Classifier with normalized training data

In [ ]:
def score(return_dict, n_neighbors, n_training_neighbors, alpha, beta, kappa, theta, n):
    nearest_pose_estimator = neighbors.NearestPoses(
        n_neighbors=n_neighbors,
        n_training_neighbors=n_training_neighbors,
        alpha=alpha,
        beta=beta,
        kappa=kappa
    )
    action_classifier = classifiers.ActionClassifier(
        nearest_pose_estimator=nearest_pose_estimator,
        theta=theta,
        n=n
    )
    action_classifier.fit(
        X_train,
        y_train,
        cache_path=f"../pickle/action_classifier_training-{str(n_training_neighbors)}.p"
    )
    y_pred = action_classifier.predict_all(X_test, verbose=False)

    result = 0
    for pred, actual in zip(y_pred, y_test):
        result += 1 if pred == actual else 0

    best_score = return_dict["best"]
    if result > best_score:
        print(f"Correctly predicted {result}/{len(y_pred)}")
        return_dict["best"] = result
        action_classifier.save_pickle("../pickle/best_action_classifier.p")


manager = multiprocessing.Manager()
return_dict = manager.dict()
return_dict["best"] = 0
workers = []

for kappa in [50, 100]:
    for n in [50, 100]:
        for theta in [0.4, 0.6]:
            for n_neighbors in [50, 100]:
                for n_training_neighbors in [2000, 5000, 20000, 40000]:
                    for alpha in [0.75]:
                        for beta in [0.6]:
                            worker = multiprocessing.Process(
                                target=
                                    score,
                                args=(
                                    return_dict,
                                    n_neighbors,
                                    n_training_neighbors,
                                    alpha,
                                    beta,
                                    kappa,
                                    theta,
                                    n
                                )
                            )
                            workers.append(worker)

num_cpu_cores = os.getenv("NUM_CPU_CORES")
if num_cpu_cores is None:
    num_cpu_cores = input("Enter the number of CPU cores this computer has (ex. '12'): ")
num_cpu_cores = max(1, int(num_cpu_cores) - 4)

num_workers_executed = num_cpu_cores
while num_workers_executed - num_cpu_cores < len(workers):
    for i in range(num_cpu_cores, max(num_workers_executed - len(workers), 0), -1):
        workers[num_workers_executed - i].start()
    for i in range(num_cpu_cores, max(num_workers_executed - len(workers), 0), -1):
        workers[num_workers_executed - i].join()
    num_workers_executed += num_cpu_cores
